### Loading all Libraries

In [76]:
import numpy as np
import pandas as pd
from skimpy import skim
pd.set_option('display.max_columns',None) 
pd.set_option('display.max_rows',800) 
import matplotlib.pyplot as plt
import seaborn as sns
import math

#from autoviz.AutoViz_Class import AutoViz_Class
# av = AutoViz_Class()
# viz = av.AutoViz('../Kaggle2/Data/training_data.csv',sep=',')

### Loading all the data files

In [38]:
campaign_train = pd.read_csv('../Kaggle2/Data/training_data.csv')
campaign_test = pd.read_csv('../Kaggle2/Data/test_data.csv')
sample_submission = pd.read_csv('../Kaggle2/Data/sample_solutions.csv')

/var/folders/bp/z_zp050d4qq14m74__f0gtnh0000gn/T/ipykernel_74523/3017861576.py:1: DtypeWarning: Columns (31,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  campaign_train = pd.read_csv('../Kaggle2/Data/training_data.csv')


In [39]:
campaign_train.head()

,general_sector,city,zip_code,specific_sector,state,contributor_type,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,politician_incumbency_count,politician_open_pos_count,politician_republican_count,contribution_democratic_sum_2010_usd,contribution_republican_sum_2010_usd,contribution_sum_2010_usd,governor_contributions_sum_2010_usd,house_and_assembly_contributions_sum_2010_usd,senate_contributions_sum_2010_usd,us_house_contributions_sum_2010_usd,us_senate_contributions_sum_2010_usd,candidacy_democratic_ratio,candidacy_republican_ratio,contribution_democratic_count_ratio,contribution_republican_count_ratio,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
0,Retired,MISSOULA,59802.0,Retired,MT,Individual,1.0,4,4,0,6,6,0,0,4,4,2,2,0,350.00,0.0,350.00,175.00,0.00,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.5,0.000000,0.0,1.0,0.5,0.5,0.0,0.0,0.0,0.0
1,Retired,CLEVELAND HTS,44118.0,Retired,OH,Individual,0.5,2,2,0,2,2,0,1,2,2,1,0,0,154.67,0.0,154.67,0.00,33.44,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.216202,0.5,1.0,0.5,0.0,0.0,0.0,0.0,0.0
2,Public Sector Unions,NEWARK,NaN,Teachers unions,NJ,Non-Individual,1.0,4,4,0,4,4,0,0,4,4,4,0,0,354.44,0.0,354.44,0.00,1154.44,-800.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,3.257082,0.0,1.0,1.0,0.0,0.0,-2.257082,0.0,0.0
3,Retired,BOZEMAN,59715.0,Retired,MT,Individual,1.0,4,4,0,4,4,0,0,4,4,2,2,0,250.00,0.0,250.00,150.00,50.00,50.0,0.0,0.0,1.0,0.0,1.0,0.0,0.6,0.200000,0.0,1.0,0.5,0.5,0.0,0.2,0.0,0.0
4,No Employer Listed or Found,ORANGE PARK,32003.0,No employer listed or discovered,FL,Individual,0.0,4,4,0,6,6,0,2,4,4,0,2,0,146.06,0.0,146.06,146.06,0.00,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.000000,0.5,1.0,0.0,0.5,0.0,0.0,0.0,0.0


**We first found values such as inf, mixed variable types while doing a high level look at the dataset. After the  cleaning of the data, those issues are now not visible anymore**

In [41]:
campaign_train.loc[campaign_train['governor_contribution_ratio']=="#NAME?",'governor_contribution_ratio']=\
                np.NaN
campaign_train.loc[campaign_train['senate_contribution_ratio']=="#NAME?",'senate_contribution_ratio']=\
                np.NaN
campaign_train.loc[campaign_train['us_senate_contribution_ratio']=="#NAME?",'us_senate_contribution_ratio']=\
                np.NaN

campaign_train.replace([np.inf, -np.inf], np.nan, inplace=True)

campaign_train.governor_contribution_ratio.fillna(0.0, inplace=True)
campaign_train.senate_contribution_ratio.fillna(0.0, inplace=True)
campaign_train.us_senate_contribution_ratio.fillna(0.0, inplace=True)

campaign_train.governor_contribution_ratio = campaign_train.governor_contribution_ratio.astype('float64')
campaign_train.senate_contribution_ratio = campaign_train.senate_contribution_ratio.astype('float64')
campaign_train.us_senate_contribution_ratio = campaign_train.us_senate_contribution_ratio.astype('float64')

campaign_train.drop_duplicates(keep='first',inplace=True)

city_state_train = campaign_train[['city','zip_code','state']].drop_duplicates(keep='first')
city_state_test = campaign_test[['city','zip_code','state']].drop_duplicates(keep='first')
city_state = pd.concat([city_state_train,city_state_test], axis=0).drop_duplicates().reset_index(drop=True)
city_state = city_state.query('zip_code.notnull() & city.notnull()')
city_state_dict = dict(zip(zip(city_state['state'],city_state['zip_code']),city_state['city']))
campaign_train['city_new'] = campaign_train.set_index(['state', 'zip_code']).index.map(city_state_dict.get)
campaign_train['city'] = np.where(campaign_train['city'].isnull() & campaign_train['zip_code'] != 0.0,
                                  campaign_train['city_new'],campaign_train['city'])
campaign_train.drop(columns=['city_new'], axis=1, inplace=True)


In [9]:
campaign_train.describe()

,zip_code,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,politician_incumbency_count,politician_open_pos_count,politician_republican_count,contribution_democratic_sum_2010_usd,contribution_republican_sum_2010_usd,contribution_sum_2010_usd,governor_contributions_sum_2010_usd,house_and_assembly_contributions_sum_2010_usd,senate_contributions_sum_2010_usd,us_house_contributions_sum_2010_usd,us_senate_contributions_sum_2010_usd,candidacy_democratic_ratio,candidacy_republican_ratio,contribution_democratic_count_ratio,contribution_republican_count_ratio,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
count,166725.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,1.704380e+05,1.704380e+05,1.704380e+05,1.704380e+05,1.704380e+05,1.704380e+05,1.704380e+05,1.704380e+05,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170248.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170438.000000,170248.000000,170438.000000
mean,39022.404121,0.594335,6.108931,3.298372,2.810559,9.766637,5.391034,4.375603,0.705799,5.077195,2.762788,2.480304,1.807825,2.314408,4.302055e+03,4.816157e+03,9.118212e+03,1.507448e+03,1.604763e+03,1.151981e+03,2.872424e+03,1.234287e+03,0.555183,0.444817,0.555424,0.444576,0.258244,0.255765,0.159344,0.555322,0.391677,0.424987,0.444678,0.163182,0.119072,0.083515
std,28765.015916,0.409621,25.736597,14.112114,14.906904,93.179851,31.162407,82.072638,2.012687,15.241222,8.514875,11.182144,4.187276,9.169623,8.351703e+04,2.637692e+05,2.807837e+05,1.180646e+05,8.384234e+04,4.492997e+04,7.025298e+04,2.838592e+04,0.467958,0.467958,0.469151,0.469151,0.405261,0.401031,0.303482,0.467394,0.407902,0.412719,0.467394,0.341783,0.430857,0.396849
min,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,-3.107367e+05,-3.674252e+05,-3.674252e+05,-2.440000e+04,-1.126210e+05,-3.790261e+05,-3.107367e+05,-2.376295e+04,0.000000,0.000000,0.000000,0.000000,-23.380614,-6.118411,0.000000,0.000000,0.000000,0.000000,0.000000,-29.388715,-6.448457,-92.603744
25%,19464.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.149100e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,30523.000000,0.666667,2.000000,2.000000,1.000000,4.000000,2.000000,1.000000,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.340900e+02,4.112000e+01,1.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.800000,0.200000,0.833333,0.166667,0.000000,0.000000,0.000000,0.800000,0.333333,0.333333,0.200000,0.000000,0.000000,0.000000
75%,63102.000000,1.000000,4.000000,3.000000,2.000000,7.000000,4.000000,3.000000,1.000000,4.000000,2.000000,2.000000,2.000000,2.000000,1.294240e+03,1.008200e+03,2.798225e+03,3.030700e+02,2.748225e+02,1.000000e+02,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000,1.000000,0.520256,0.481066,0.200000,1.000000,0.800000,1.000000,1.000000,0.120159,0.000000,0.000000
max,99999.000000,1.000000,2026.000000,1284.000000,1014.000000,31652.000000,3990.000000,31648.000000,140.000000,829.000000,543.000000,608.000000,296.000000,584.000000,1.468935e+07,9.500888e+07,9.503992e+07,4.472358e+07,2.229992e+07,1.375416e+07,1.220771e+07,6.78302

In [42]:
skim(campaign_train)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 170438 │ │ float64     │ 24    │                                                          │
│ │ Number of columns │ 41     │ │ int64       │ 12    │                                                          │
│ └───────────────────┴────────┘ │ string      │ 5     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name             ┃ NA    ┃ NA %  ┃ mean   ┃ sd      ┃ p0       ┃ p25    ┃ p75   ┃ p100     ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩  │
│ │ zip_code                │  3700 │   2.2 │  39000 │   29000 │        0 │  19000 │ 63000 │   100000 │ ▅█▂▂▃▃ │  │
│ │ winner_ratio            │     0 │     0 │   0.59 │    0.41 │        0 │      0 │     1 │        1 │ ▅▁ ▂▂█ │  │
│ │ candidacy_count         │     0 │     0 │    6.1 │      26 │        2 │      2 │     4 │     2000 │   █    │  │
│ │ candidacy_democratic    │     0 │     0 │    3.3 │      14 │        0 │      0 │     3 │     1300 │   █    │  │
│ │ candidacy_republican    │     0 │     0 │    2.8 │      15 │        0 │      0 │     2 │     1000 │   █    │  │
│ │ contribution_count      │     0 │     0 │    9.8 │      93 │        2 │      2 │     7 │    32000 │   █    │  │
│ │ contribution_democra    │     0 │     0 │    5.4 │      31 │        0 │      0 │     4 │     4000 │   █    │  │
│ │ contribution_republi    │     0 │     0 │    4.4 │      82 │        0 │      0 │     3 │    32000 │   █    │  │
│ │ politician_challenge    │     0 │     0 │   0.71 │       2 │        0 │      0 │     1 │      140 │   █    │  │
│ │ politician_count        │     0 │     0 │    5.1 │      15 │        2 │      2 │     4 │      830 │   █    │  │
│ │ politician_democrati    │     0 │     0 │    2.8 │     8.5 │        0 │      0 │     2 │      540 │   █    │  │
│ │ politician_incumbenc    │     0 │     0 │    2.5 │      11 │        0 │      0 │     2 │      610 │   █    │  │
│ │ politician_open_pos_    │     0 │     0 │    1.8 │     4.2 │        0 │      0 │     2 │      300 │   █    │  │
│ │ politician_republica    │     0 │     0 │    2.3 │     9.2 │        0 │      0 │     2 │      580 │   █    │  │
│ │ contribution_democra    │     0 │     0 │   4300 │   84000 │  -310000 │      0 │  1300 │ 15000000 │   █    │  │
│ │ contribution_republi    │     0 │     0 │   4800 │  260000 │  -370000 │      0 │  1000 │ 95000000 │   █    │  │
│ │ contribution_sum_201    │     0 │     0 │   9100 │  280000 │  -370000 │    310 │  2800 │ 95000000 │   █    │  │
│ │ governor_contributio    │     0 │     0 │   1500 │  120000 │   -24000 │      0 │   300 │ 45000000 │   █    │  │
│ │ house_and_assembly_c    │     0 │     0 │   1600 │   84000 │  -110000 │      0 │   270 │ 22000000 │   █    │  │
│ │ senate_contributions    │     0 │     0 │   1200 │   45000 │  -380000 │      0 │   100 │ 14000000 │   █    │  │
│ │ us_house_contributio    │     0 │     0 │   2900 │   70000 │  -310000 │      0 │     0 │ 12000000 │   █    │  │
│ │ us_senate_contributi    │     0 │     0 │   1200 │  

In [13]:
skim(campaign_test)

/Users/sayankrswar/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 115232 │ │ float64     │ 24    │                                                          │
│ │ Number of columns │ 41     │ │ int64       │ 12    │                                                          │
│ └───────────────────┴────────┘ │ string      │ 5     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name           ┃ NA      ┃ NA %   ┃ mean   ┃ sd      ┃ p0      ┃ p25   ┃ p75   ┃ p100      ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩  │
│ │ zip_code              │    2500 │    2.2 │  39000 │   29000 │       0 │ 19000 │ 63000 │    100000 │ ▅█▃▂▃▃ │  │
│ │ winner_ratio          │  120000 │    100 │    nan │     nan │     nan │   nan │   nan │       nan │        │  │
│ │ candidacy_count       │       0 │      0 │    6.1 │      32 │       2 │     2 │     4 │      5000 │   █    │  │
│ │ candidacy_democratic  │       0 │      0 │    3.3 │      16 │       0 │     0 │     3 │      2100 │   █    │  │
│ │ candidacy_republican  │       0 │      0 │    2.8 │      18 │       0 │     0 │     2 │      2900 │   █    │  │
│ │ contribution_count    │       0 │      0 │    9.9 │     110 │       2 │     2 │     7 │     33000 │   █    │  │
│ │ contribution_democra  │       0 │      0 │    5.6 │      55 │       0 │     0 │     4 │     15000 │   █    │  │
│ │ contribution_republi  │       0 │      0 │    4.3 │      62 │       0 │     0 │     3 │     18000 │   █    │  │
│ │ politician_challenge  │       0 │      0 │   0.73 │     4.8 │       0 │     0 │     1 │      1400 │   █    │  │
│ │ politician_count      │       0 │      0 │    5.1 │      19 │       2 │     2 │     4 │      3500 │   █    │  │
│ │ politician_democrati  │       0 │      0 │    2.8 │      10 │       0 │     0 │     2 │      1500 │   █    │  │
│ │ politician_incumbenc  │       0 │      0 │    2.5 │      11 │       0 │     0 │     2 │       640 │   █    │  │
│ │ politician_open_pos_  │       0 │      0 │    1.8 │     6.6 │       0 │     0 │     2 │      1500 │   █    │  │
│ │ politician_republica  │       0 │      0 │    2.3 │      11 │       0 │     0 │     2 │      2000 │   █    │  │
│ │ contribution_democra  │       0 │      0 │   7900 │ 1000000 │ -250000 │     0 │  1300 │ 340000000 │   █    │  │
│ │ contribution_republi  │       0 │      0 │   8100 │  990000 │ -130000 │     0 │  1000 │ 310000000 │   █    │  │
│ │ contribution_sum_201  │       0 │      0 │  16000 │ 2000000 │ -250000 │   300 │  2800 │ 650000000 │   █    │  │
│ │ governor_contributio  │       0 │      0 │   2100 │  240000 │  -48000 │     0 │   290 │  76000000 │   █    │  │
│ │ house_and_assembly_c  │       0 │      0 │   1400 │   37000 │ -250000 │     0 │   270 │   7000000 │   █    │  │
│ │ senate_contributions  │       0 │      0 │   1200 │   42000 │  -88000 │     0 │    98 │  12000000 │   █    │  │
│ │ us_house_contributio  │       0 │      0 │   5500 │ 1000000 │  -31000 │     0 │     0 │ 340000000 │   █    │  │
│ │ us_senate_contributi  │       0 │      0 │   5400 │ 

### EDA on the categprical columns, especially on City, Zipcode columns for which there are many NULLS.

In [45]:
campaign_train[campaign_train.city.isnull()].head()

,general_sector,city,zip_code,specific_sector,state,contributor_type,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,politician_incumbency_count,politician_open_pos_count,politician_republican_count,contribution_democratic_sum_2010_usd,contribution_republican_sum_2010_usd,contribution_sum_2010_usd,governor_contributions_sum_2010_usd,house_and_assembly_contributions_sum_2010_usd,senate_contributions_sum_2010_usd,us_house_contributions_sum_2010_usd,us_senate_contributions_sum_2010_usd,candidacy_democratic_ratio,candidacy_republican_ratio,contribution_democratic_count_ratio,contribution_republican_count_ratio,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
209,Uncoded,NaN,NaN,Uncoded,NJ,Individual,0.0,2,0,2,2,0,2,2,2,0,0,0,2,0.00,80.00,80.00,0.0,80.00,0.00,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
233,Insurance,NaN,NaN,Property & casualty insurance,KS,Non-Individual,1.0,2,0,2,2,0,2,0,2,0,2,0,2,0.00,253.26,253.26,0.0,253.26,0.00,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
270,Uncoded,NaN,NaN,Uncoded,NJ,Individual,1.0,2,2,0,2,2,0,0,2,2,2,0,0,936.04,0.00,936.04,0.0,0.00,936.04,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
555,No Employer Listed or Found,NaN,NaN,No employer listed or discovered,NY,Individual,0.0,2,2,0,2,2,0,0,2,2,0,2,0,200.00,0.00,200.00,0.0,200.00,0.00,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
639,Uncoded,NaN,NaN,Uncoded,NM,Individual,1.0,2,0,2,2,0,2,2,2,0,0,0,2,0.00,242.46,242.46,0.0,242.46,0.00,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [43]:
#might drop this rows will come back later
print(campaign_train[campaign_train.general_sector=='Uncoded']['city'].isnull().sum())

#checking the count of uncoded cases with null city
print(campaign_train[campaign_train.general_sector!='Uncoded']['city'].isnull().sum())

477
812


In [44]:
cols = ['general_sector','specific_sector','contributor_type']+list(campaign_train.columns[7:])


campaign_train[(campaign_train.general_sector=='Uncoded') & (campaign_train['city'].isnull())]\
.sort_values(by=['state']+cols)

campaign_test[(campaign_test.general_sector=='Uncoded') & (campaign_test['city'].isnull())]\
.sort_values(by=['state']+cols)

campaign_train[(campaign_train.state=='KS')].sort_values(by=cols)

df_temp = campaign_train[campaign_train[['general_sector','specific_sector','state','winner_ratio']]\
               .duplicated(keep=False)]\
               .sort_values(by=['general_sector','specific_sector','state','winner_ratio'])

campaign_test[(campaign_test.state=='KS')]

campaign_test[(campaign_test.general_sector=='Insurance') & (campaign_test.state=='KS')]

campaign_train[campaign_train[cols].duplicated(keep=False)]\
.sort_values(by=cols)

campaign_train[['city','zip_code','state','city_new']].query("zip_code.notnull() & city.isnull()").head(500)


### EDA on Numerical Variables

In [49]:
campaign_train.head(100)

,general_sector,city,zip_code,specific_sector,state,contributor_type,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,politician_incumbency_count,politician_open_pos_count,politician_republican_count,contribution_democratic_sum_2010_usd,contribution_republican_sum_2010_usd,contribution_sum_2010_usd,governor_contributions_sum_2010_usd,house_and_assembly_contributions_sum_2010_usd,senate_contributions_sum_2010_usd,us_house_contributions_sum_2010_usd,us_senate_contributions_sum_2010_usd,candidacy_democratic_ratio,candidacy_republican_ratio,contribution_democratic_count_ratio,contribution_republican_count_ratio,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
0,Retired,MISSOULA,59802.0,Retired,MT,Individual,1.000000,4,4,0,6,6,0,0,4,4,2,2,0,350.00,0.00,350.00,175.00,0.00,0.00,0.00,0.00,1.000000,0.000000,1.000000,0.000000,0.500000,0.000000,0.000000,1.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000
1,Retired,CLEVELAND HTS,44118.0,Retired,OH,Individual,0.500000,2,2,0,2,2,0,1,2,2,1,0,0,154.67,0.00,154.67,0.00,33.44,0.00,0.00,0.00,1.000000,0.000000,1.000000,0.000000,0.000000,0.216202,0.500000,1.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Public Sector Unions,NEWARK,NaN,Teachers unions,NJ,Non-Individual,1.000000,4,4,0,4,4,0,0,4,4,4,0,0,354.44,0.00,354.44,0.00,1154.44,-800.00,0.00,0.00,1.000000,0.000000,1.000000,0.000000,0.000000,3.257082,0.000000,1.000000,1.000000,0.000000,0.000000,-2.257082,0.000000,0.000000
3,Retired,BOZEMAN,59715.0,Retired,MT,Individual,1.000000,4,4,0,4,4,0,0,4,4,2,2,0,250.00,0.00,250.00,150.00,50.00,50.00,0.00,0.00,1.000000,0.000000,1.000000,0.000000,0.600000,0.200000,0.000000,1.000000,0.500000,0.500000,0.000000,0.200000,0.000000,0.000000
4,No Employer Listed or Found,ORANGE PARK,32003.0,No employer listed or discovered,FL,Individual,0.000000,4,4,0,6,6,0,2,4,4,0,2,0,146.06,0.00,146.06,146.06,0.00,0.00,0.00,0.00,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.500000,1.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000
5,Uncoded,JACKSON,27845.0,Uncoded,NC,Individual,1.000000,2,2,0,4,4,0,0,2,2,0,0,0,854.50,0.00,854.50,854.50,0.00,0.00,0.00,0.00,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Lawyers & Lobbyists,PORTLAND,97221.0,Attorneys & law firms,US,Individual,1.000000,7,7,0,14,14,0,0,4,4,4,0,0,6433.70,0.00,6433.70,0.00,0.00,0.00,5933.70,500.00,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.922284,0.077716
7,Lawyers & Lobbyists,SWANQUARTER,27885.0,Attorneys & law firms,NC,Individual,0.285714,7,0,7,8,0,8,1,6,0,2,3,6,0.00,2330.49,2330.49,2057.46,273.03,0.00,0.00,0.00,0.000000,1.000000,0.000000,1.000000,0.882844,0.117156,0.166667,0.000000,0.333333,0.500000,1.000000,0.000000,0.000000,0.000000
8,Lawyers & Lobbyists,ALEXANDRIA,22307.0,Lobbyists & public relations,US,Individual,0.750000,12,0,12,14,0,14,0,11,0,8,3,11,0.00,6651.90,6651.90,0.00,0.00,0.00,3880.42,2771.48,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.727273,0.272727,1.000000,0.000000,0.583355,0.416645
9,Retired,PUEBLO,81008.0,Retired,CO,Individual,0.857143,7,3,4,8,3,5,0,6,2,2,4,4,130.68,127.06,257.74,23.40,165.26,69.08,0.00,0.00,0.428571,0.571429,0.375000,0.625000,0.090789,0.641189,0.000000,0.333333,0.333333,0.666667,0.666667,0.268022,0.000000,0.000000


**Because there are missing values in the train dataset for house_and_assembly_contribution_ratio and us_house_contribution_ratio after skimming the data set thus we are imputing this columns with 0.0**

In [54]:
campaign_train.house_and_assembly_contribution_ratio.fillna(0.0, inplace=True)
campaign_train.us_house_contribution_ratio.fillna(0.0, inplace=True)

print(campaign_train.house_and_assembly_contribution_ratio.isnull().sum())
print(campaign_train.us_house_contribution_ratio.isnull().sum())


0
0


**Similarly there are missing values in the test dataset for governor_contribution_ratio, house_and_assembly_contribution_ratio and us_house_contribution_ratio,senate_contribution_ratio, us_senate_contribution_ratio after skimming the data set thus we are imputing this columns with 0.0**

In [57]:
campaign_test.house_and_assembly_contribution_ratio.fillna(0.0, inplace=True)
campaign_test.us_house_contribution_ratio.fillna(0.0, inplace=True)
campaign_test.senate_contribution_ratio.fillna(0.0, inplace=True)
campaign_test.us_senate_contribution_ratio.fillna(0.0, inplace=True)
campaign_test.governor_contribution_ratio.fillna(0.0, inplace=True)

print(campaign_test.house_and_assembly_contribution_ratio.isnull().sum())
print(campaign_test.us_house_contribution_ratio.isnull().sum())
print(campaign_test.senate_contribution_ratio.isnull().sum())
print(campaign_test.us_senate_contribution_ratio.isnull().sum())
print(campaign_test.governor_contribution_ratio.isnull().sum())


0
0
0
0
0


### Trying to see correlation among the numerical columns

1. Majorly the numerical columns does not have a high correlation with our dependent variable. However few columns such as politician_incumbency_ratio, politician_challenger_ratio has very high correaltion with the dependent variable.

In [139]:
campaign_train[campaign_train.columns[6:]].corr()

,winner_ratio,candidacy_count,candidacy_democratic_count,candidacy_republican_count,contribution_count,contribution_democratic_count,contribution_republican_count,politician_challenger_count,politician_count,politician_democratic_count,politician_incumbency_count,politician_open_pos_count,politician_republican_count,contribution_democratic_sum_2010_usd,contribution_republican_sum_2010_usd,contribution_sum_2010_usd,governor_contributions_sum_2010_usd,house_and_assembly_contributions_sum_2010_usd,senate_contributions_sum_2010_usd,us_house_contributions_sum_2010_usd,us_senate_contributions_sum_2010_usd,candidacy_democratic_ratio,candidacy_republican_ratio,contribution_democratic_count_ratio,contribution_republican_count_ratio,governor_contribution_ratio,house_and_assembly_contribution_ratio,politician_challenger_ratio,politician_democratic_ratio,politician_incumbency_ratio,politician_open_pos_ratio,politician_republican_ratio,senate_contribution_ratio,us_house_contribution_ratio,us_senate_contribution_ratio
winner_ratio,1.000000,0.057517,0.045788,0.055956,0.022150,0.023108,0.016374,-0.120256,0.060557,0.043935,0.112375,-0.024386,0.059857,0.014308,0.006828,0.010670,0.002317,0.007207,0.009127,0.016830,0.011925,-0.023029,0.023029,-0.023062,0.023062,-0.142966,0.086951,-0.334608,-0.022924,0.510879,-0.259842,0.022924,0.083350,-0.007429,0.009808
candidacy_count,0.057517,1.000000,0.880111,0.893303,0.593450,0.760105,0.385157,0.465891,0.968697,0.833622,0.955862,0.729347,0.836015,0.401091,0.223881,0.329616,0.120804,0.248415,0.265502,0.461128,0.229839,-0.007739,0.007739,-0.007805,0.007805,-0.048661,0.009979,-0.022015,-0.007062,0.073962,-0.051816,0.007062,0.012841,0.033771,0.004709
candidacy_democratic_count,0.045788,0.880111,1.000000,0.572818,0.467715,0.873558,0.199329,0.476234,0.839580,0.963689,0.824348,0.602204,0.500624,0.491373,0.077385,0.218851,0.015947,0.160299,0.180841,0.433683,0.195941,0.144865,-0.144865,0.145007,-0.145007,-0.045898,0.004137,-0.012910,0.144322,0.062456,-0.048044,-0.144322,0.010286,0.027480,0.006980
candidacy_republican_count,0.055956,0.893303,0.572818,1.000000,0.581806,0.485330,0.476267,0.353512,0.877628,0.526931,0.869888,0.689113,0.969437,0.227304,0.313268,0.361895,0.193470,0.277134,0.287188,0.385572,0.211321,-0.150502,0.150502,-0.150751,0.150751,-0.040562,0.013313,-0.025786,-0.148820,0.068569,-0.043978,0.148820,0.012432,0.032292,0.001522
contribution_count,0.022150,0.593450,0.467715,0.581806,1.000000,0.502403,0.944575,0.339445,0.561575,0.434596,0.535440,0.441563,0.529855,0.272663,0.830230,0.861023,0.786877,0.672822,0.559847,0.339387,0.155879,-0.000736,0.000736,-0.000768,0.000768,-0.022797,-0.005187,-0.007859,-0.000369,0.031100,-0.023429,0.000369,-0.001791,0.028232,0.008419
contribution_democratic_count,0.023108,0.760105,0.873558,0.485330,0.502403,1.000000,0.190702,0.434404,0.712723,0.831596,0.703627,0.488815,0.412431,0.562993,0.079863,0.242482,0.011960,0.170711,0.145527,0.541856,0.221578,0.110987,-0.110987,0.111816,-0.111816,-0.040093,-0.014619,-0.005679,0.110599,0.039572,-0.032837,-0.110599,-0.006819,0.049313,0.019185
contribution_republican_count,0.016374,0.385157,0.199329,0.476267,0.944575,0.190702,1.000000,0.220443,0.366959,0.177660,0.340741,0.315722,0.444965,0.095800,0.912265,0.885480,0.888827,0.699060,0.580358,0.179579,0.092843,-0.042976,0.042976,-0.043328,0.043328,-0.010659,-0.000339,-0.006766,-0.042413,0.020283,-0.014132,0.042413,0.000556,0.013329,0.002274
politician_challenger_count,-0.120256,0.465891,0.476234,0.353512,0.339445,0.434404,0.220443,1.000000,0.520072,0.523686,0.363338,0.440749,0.378141,0.271482,0.222407,0.289680,0.162385,0.250791,0.259968,0.265448,0.142313,0.024212,-0.024212,0.024559,-0.024559,-0.066452,-0.002241,0.420442,0.024102,-0.120206,-0.172675,-0.024102,-0.001125,0.054869,0.033643
politician_count,0.060557,0.968697,0.839580,0.877628,0.561575,0.712723,0.366959,0.520072,1.000000,0.850366,0.959347,0.805305,0.872497,0.376753,0.217918,0.316775,0.120584,0.238728,0.245717,0.434145,0.231030,-0.00

### Lets Build Some Model and See how scores are varying

1. Because there is a lot of missing values in City and Zip code columns therefore we are not going to use those columns in the train/test models.
2. Also there are many values which are showing as Uncoded for general_sector, specific_sector columns. This columns can cause the model to get confused and thus there is a huge chance that the model will perform poorly.

Note that this is the clean code with final model

In [73]:
from catboost import CatBoostRegressor
from catboost import Pool
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import KFold, train_test_split, cross_val_score, RandomizedSearchCV, ShuffleSplit, RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

In [107]:
cols_X_train = ['state']+list(campaign_train.columns[7:])
cols_y_train = ['winner_ratio']

In [98]:
X = campaign_train[cols_X_train]
y = campaign_train[cols_y_train]

categoricalcolumns = X.select_dtypes(include=["object"]).columns.tolist()
cat_features = ['state']

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7,random_state= 55)

train_data = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_features
                 )
test_data = Pool(data=X_test,
                  label=y_test,
                  cat_features=cat_features
                 )

In [99]:
cat_model = CatBoostRegressor(loss_function='RMSE', depth=12, iterations = 400, \
                              learning_rate=0.1,silent=True)
cat_model.fit(train_data,plot=True)
y_predict= cat_model.predict(X_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [100]:
#RMSE
Rmse_test = math.sqrt(mean_squared_error(y_test,y_predict))

#R2 Score
r2_test = r2_score(y_test,y_predict)

# Adjusted R2 Score
n = X_train.shape[0] # total no of datapoints
p = X_train.shape[1] # total no of independent features
adj_r2_test = 1-(1-r2_test)*(n-1)/(n-p-1)

#print results
print("Evaluation on test data")
print("RMSE: {:.2f}".format(Rmse_test))
print("R2: {:.2f}".format(r2_test))
print("Adjusted R2: {:.2f}".format(adj_r2_test))
print("Train Data RMSE:", math.sqrt(mean_squared_error(Y, model_cat_final.predict(X_cat_final))))

Evaluation on test data
RMSE: 0.29
R2: 0.51
Adjusted R2: 0.51
Train Data RMSE: 0.27601131645490684


### Final Training

In [111]:
X_final = campaign_train[cols_X_train]
y_final = campaign_train[cols_y_train]

cat_model_final= CatBoostRegressor(iterations=400, depth=12, learning_rate=0.1, loss_function='RMSE',silent=True)
cat_model_final.fit(X_final, y_final, cat_features=[0],plot=True)

print("Train RMSE", math.sqrt(mean_squared_error(Y, cat_model_final.predict(X_final))))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Train RMSE 0.2667288143264672


In [123]:
X_Test = campaign_test[cols_X_train]
cat_final_campaign_test = cat_model_final.predict(X_Test)

df_cat_final_campaign_test = pd.DataFrame(cat_final_campaign_test,columns=['winner_ratio'])
df_cat_final_campaign_test['winner_ratio'] = np.where(df_cat_final_campaign_test['winner_ratio']<0, 0.0,
                                      df_cat_final_campaign_test['winner_ratio'])

In [125]:
df_cat_final_campaign_test.to_csv("submission_SKS_1.csv", index_label='index')